# Formulation

Let i, j, k be order i, carrier j, plant k.<br>
Let $X(i,j,k)$ be a float variable representing the amount of order i sent on carrier j to plant k<br>
Let $Y(i, k)$ be a binary variable representing whether order i is shipped to plant k

Let $S_i$ be the order_weight[i]<br>
Let $CR_j$ be carrier_rate[j]; let $CC_j$ be carrier_capacity[j] <br>
Let $PR_k$ be plant_rate[k]; $PC_k$ be the plant_capacity[k]<br>


**Objective**: <br>
Minimise $ = \sum_{(i, j, k)} (CR_j + PR_k)* X(i, j, k) + \sum_{i} 5*(S_i - \sum_{(j, k)}X(i,j,k))$


**Constraints**:<br>
1. Order-Plant:<br>
Each order is shipped to only one plant.<br>
For each i, $\sum_{k} Y(i, k) = 1$<br>
2. Total Order Weight<br>
Order Weight is not exceeded.<br>
For each i and k, $\sum_{j} X(i, j, k) <= Y(i, k)*S_i$<br>
==> Along with 1, we have $\sum_{(j, k)} X(i, j, k) <= S_i$ for each i.<br>
3. Carrier Capacity:<br>
Each carrier is carrying weights under carrier capacity.<br>
For each j, $\sum_{(i, k)} X(i, j, k)<=CC_j$ where the choice of k is subject to j.<br>
4. Plant Capacity:<br>
Each plant is being delivered weights under plant capacity.<br>
For each k, $\sum_{(i, j)}X(i, j, k)<= PC_k$ where the choice of j is subject to k<br>

**Extra Credit**:<br>
M = 100 is the min order num at full.<br>
*Additional Variable*:<br>
Let $Z(i)$ be a binary variable representing whether order i is shipped at full<br>
*Additional Constraints*:<br>
1. Number of Orders at Full<br>
$\sum_{i} Z(i) >= M$ <br>
2. Orders Lower Bound<br>
For each i, $\sum_{(j, k)} X(i, j, k) >= Z(i)*S_i$

# Code

In [12]:
%pip install gurobipy

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
%cd /content/drive/My Drive/Spring_2024/IEOR4004/Projects/Project_5

/content/drive/My Drive/Spring_2024/IEOR4004/Projects/Project_5


In [15]:
from gurobipy import GRB
import gurobipy as gp
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
from collections import defaultdict
from copy import deepcopy

In [16]:
class lp():
    def __init__(self, N = None, J = None, gp_params = None, min_order_at_full = 0):
        self.N = N
        self.J = J
        self.carrier_to_plant, self.plant_to_carrier, self.plant_capacity, self.plant_rate, self.carrier_capacity, self.carrier_rate, self.order_weight = self.generate_dict()
        if gp_params:
            self.env = gp.Env(params=gp_params)
        else:
            self.env = None
        self.min_order_at_full = min_order_at_full

    def generate_dict(self):
        print('\n------Generating Dicts------')
        orderlist_full = pd.read_excel('Project_5.xlsx', sheet_name = 'OrderList')
        carriers_full = pd.read_excel('Project_5.xlsx', sheet_name = 'Carriers')
        plants = pd.read_excel('Project_5.xlsx', sheet_name = 'Plants')
        plantports = pd.read_excel('Project_5.xlsx', sheet_name='PlantPorts')

        if self.N:
            orderlist = orderlist_full[:self.N]
        else:
            self.N = len(orderlist_full)
            orderlist = orderlist_full

        if self.J:
            carriers = carriers_full[:self.J]
        else:
            self.J = len(carriers_full)
            carriers = carriers_full

        carrier_to_plant = defaultdict(list)
        for j in carriers.index:
            port_j = carriers['dest_port'].loc[j]
            plant_js = plantports['Plant Code'].loc[plantports['Port'] == port_j].tolist()
            for plant in plant_js:
                carrier_to_plant[j].append(int(plant[-2:]))

        plant_to_carrier = defaultdict(list)
        for k in range(len(plantports)):
            plant = plantports['Plant Code'].loc[k]
            plant_code = int(plant[-2:])
            port = plantports['Port'].loc[k]
            carrier_ks = carriers.index[carriers['dest_port'] == port].tolist()
            plant_to_carrier[plant_code].extend(carrier_ks)

        plant_capacity = {int(ID[-2:]): cap*1000 for ID, cap in zip(plants['WH'], plants['Capacity (1000s)'])}

        plant_rate = defaultdict(float)
        for k in plants.index:
            plant_rate[k] = plants.loc[k]['Per unit cost']

        carrier_capacity = defaultdict(float)
        for j in carriers.index:
            carrier_capacity[j] = carriers.loc[j]['capacity']

        carrier_rate = defaultdict(float)
        for j in carriers.index:
            carrier_rate[j] = carriers.loc[j]['rate']

        order_weight = defaultdict(float)
        for i in orderlist.index:
            order_weight[i] = orderlist.loc[i]['Weight']

        return carrier_to_plant, plant_to_carrier, plant_capacity, plant_rate, carrier_capacity, carrier_rate, order_weight

    def lp_model(self, time_limit = 10800):
        if self.env:
            m = gp.Model(f'm_{self.min_order_at_full}', env = self.env)
        else:
            m = gp.Model(f'm_{self.min_order_at_full}')

        print('****** Creating Variable List ******')
        xvar = {}
        for i in tqdm(self.order_weight.keys()):
            order_w = self.order_weight[i]
            for j in self.carrier_capacity.keys():
                carrier_c = self.carrier_capacity[j]
                ks = self.carrier_to_plant[j]
                for k in ks:
                    xvar[i,j,k] = m.addVar(lb = 0.0, ub = max(order_w, carrier_c), vtype = GRB.CONTINUOUS, name = f'X_{i},{j},{k}')

        yvar = {}
        for i in tqdm(self.order_weight.keys()):
            for k in self.plant_capacity.keys():
                yvar[i, k] = m.addVar(vtype = GRB.BINARY, name = f'Y_{i},{k}')

        order_part = {}
        for i in tqdm(self.order_weight.keys()):
            order_w = self.order_weight[i]
            order_part[i] = m.addVar(lb = 0.0, ub = order_w, vtype = GRB.CONTINUOUS, name = f'OrderPart_{i}')
            m.addConstr(sum(xvar[i,j,k] for j in self.carrier_capacity.keys() for k in self.carrier_to_plant[j]) == order_part[i], name = f'OrderPartMatch_{i}')

        print('****** Setting Objective ******')
        total_cost = m.addVar(name='total_cost')
        m.setObjective(total_cost, GRB.MINIMIZE)

        sum_of_rates = sum((self.carrier_rate[j] + self.plant_rate[k]) * xvar[i,j,k] for i, j, k in tqdm(xvar.keys()))
        cost_of_not_shipped = sum(5*(self.order_weight[i] - order_part[i]) for i in tqdm(self.order_weight.keys()))
        m.addConstr(total_cost == sum_of_rates + cost_of_not_shipped, name = 'Total_Cost')

        print('****** Setting Constraints ******')
        for i in tqdm(self.order_weight.keys()):
            m.addConstr(
                sum(yvar[i, k] for k in self.plant_capacity.keys()) == 1,
                name=f'OrderPlant_{i}'
            )

        for i in tqdm(self.order_weight.keys()):
            order_w = self.order_weight[i]
            for k in self.plant_capacity.keys():
                m.addConstr(
                    sum(xvar[i, j, k] for j in self.plant_to_carrier[k]) <= order_w * yvar[i,k],
                    name = f'OrderWeight_{i},{k}'
                )

        for j in tqdm(self.carrier_capacity.keys()):
            m.addConstr(
                sum(xvar[i, j, k] for i in self.order_weight.keys() for k in self.carrier_to_plant[j]) <= self.carrier_capacity[j],
                name=f'Carriers_{j}'
            )

        for k in tqdm(self.plant_capacity.keys()):
            m.addConstr(
                sum(xvar[i, j, k] for i in self.order_weight.keys() for j in self.plant_to_carrier[k]) <= self.plant_capacity[k],
                name=f'Plants_{k}'
            )

        zvar = {}
        if self.min_order_at_full != 0:
            for i in tqdm(self.order_weight.keys()):
                zvar[i] = m.addVar(vtype = GRB.BINARY, name = f'Z_{i}')

            m.addConstr(
                sum(zvar[i] for i in self.order_weight.keys()) >= self.min_order_at_full, name = 'Num_at_Full'
            )
            for i in tqdm(self.order_weight.keys()):
                order_w = self.order_weight[i]
                m.addConstr(
                    sum(xvar[i, j, k] for j in self.carrier_capacity.keys() for k in self.carrier_to_plant[j]) >= zvar[i]*order_w, name = f'Order_{i}_LowerBound'
                )


        try:
            m.update()
            print('****** Optimising ******')
            m.setParam('TimeLimit', time_limit)
            m.setParam('NodefileStart', 10)
            m.setParam('Threads', 8)
            m.optimize()
        except gp.GurobiError as e:
            print(f'Error code {e.errno}: {e}', file=sys.stderr)
        except AttributeError:
            print('Encountered an attribute error', file=sys.stderr)

        return m, xvar, yvar, zvar, total_cost, order_part

    def summary(self, m, xvar, yvar, zvar, total_cost, order_part):
        carrier_weight = defaultdict(float)
        plant_weight = defaultdict(float)
        for i, j, k in xvar.keys():
            carrier_weight[j] += xvar[i,j,k].x
            plant_weight[k] += xvar[i,j,k].x

        with open(f'logistics_{self.N}_{self.J}.txt', 'w') as file:
            file.write(f'Total Cost: {total_cost.x} \n')
            print(f'Total Cost: {total_cost.x} \n')

            if m.status == gp.GRB.OPTIMAL or m.status == gp.GRB.SUBOPTIMAL:
                # Get the optimization gap
                mip_gap = m.MIPGap
                file.write(f'Optimization Gap: {mip_gap:.2%} \n')
                print(f'Optimization Gap: {mip_gap:.2%} \n')

            file.write(f'Require {self.min_order_at_full} orders to be shipped in full \n')
            print(f'Require {self.min_order_at_full} orders to be shipped in full \n')

            cnt = 0
            for i in order_part.keys():
                if self.order_weight[i] - order_part[i].x < 1e-6:
                    cnt += 1
            file.write(f'Actual {cnt} orders shipped in full \n')
            print(f'Actual {cnt} orders shipped in full \n')

            cnt = 0
            for j in self.carrier_capacity.keys():
                if self.carrier_capacity[j] - carrier_weight[j] <= 1e-6:
                    cnt += 1
            file.write(f'{cnt} carriers used at full capacity \n')
            print(f'{cnt} carriers used at full capacity \n')

            cnt = 0
            for j in self.plant_capacity.keys():
                if self.plant_capacity[j] - plant_weight[j] <= 1e-6:
                    cnt += 1
            file.write(f'{cnt} plants used at full capacity \n')
            print(f'{cnt} plants used at full capacity \n')

            file.write('\n')
            file.write('Order-Plant Pair:\n')
            for i, k in yvar.keys():
                if yvar[i,k].x > 0:
                    file.write(f'Order{i} is shipping to Plant {k}')

        return


    def write_dat_file(self, m, xvar):
        order_part = defaultdict(int)
        with open(f'logistics_{self.N}_{self.J}.dat', 'w') as file:
            file.write(f"{len(self.order_weight)} {len(self.carrier_rate)} {len(self.plant_rate)}\n")
            for i,j,k in tqdm(xvar.keys()):
                file.write(f"{i} {j} {k} {xvar[i, j, k].x}\n")
                order_part[i] += xvar[i, j, k].x
            file.write("END\n")
        return

## Test Code for N = 10 and J = 10

In [17]:
# Type In Your Own gurobi.lic
params = {
    "WLSACCESSID": 'YOUR WLS ID',
    "WLSSECRET": 'YOUR WLS SECRET',
    "LICENSEID": int('YOUR LICENSE'),
}
lp_instance = lp(N = 10, J = 10, gp_params = params, min_order_at_full = 0)
m, xvar, yvar, zvar, cost, order_part = lp_instance.lp_model()


------Generating Dicts------
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2482374
Academic license 2482374 - for non-commercial use only - registered to zt___@columbia.edu
****** Creating Variable List ******


100%|██████████| 10/10 [00:00<00:00, 8253.25it/s]


****** Setting Objective ******


100%|██████████| 10/10 [00:00<00:00, 14018.40it/s]


****** Setting Constraints ******


100%|██████████| 19/19 [00:00<00:00, 9816.68it/s]

****** Optimising ******
Set parameter TimeLimit to value 10800
Set parameter NodefileStart to value 10
Set parameter Threads to value 8
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2482374 - for non-commercial use only - registered to zt___@columbia.edu
Optimize a model with 240 rows, 461 columns and 1701 nonzeros
Model fingerprint: 0x2ea650d5
Variable types: 271 continuous, 190 integer (190 binary)
Coefficient statistics:
  Matrix range     [8e-01, 9e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 2316.5000000
Presolve removed 83 rows and 71 columns
Presolve time: 0.00s
Presolved: 157 rows, 390 columns, 1000 nonzeros
Variable types: 260 continuous, 130 integer (130 binary)

Root relaxati

     0     2  520.64459    0    2  520.74904  520.64459  0.02%     -    0s
H    4     4                     520.6828510  520.64459  0.01%   4.5    0s

Cutting planes:
  Cover: 1
  Implied bound: 5
  Flow cover: 4
  Relax-and-lift: 2

Explored 5 nodes (162 simplex iterations) in 0.13 seconds (0.01 work units)
Thread count was 8 (of 8 available processors)

Solution count 8: 520.683 520.749 520.868 ... 2316.5

Optimal solution found (tolerance 1.00e-04)
Best objective 5.206828510191e+02, best bound 5.206445877972e+02, gap 0.0073%


In [18]:
lp_instance.summary(m, xvar, yvar, zvar, cost, order_part)

Total Cost: 520.6828510190655 

Optimization Gap: 0.01% 

Require 0 orders to be shipped in full 

Actual 10 orders shipped in full 

1 carriers used at full capacity 

0 plants used at full capacity 



In [19]:
lp_instance.write_dat_file(m, xvar)

100%|██████████| 260/260 [00:00<00:00, 161152.51it/s]


## N = 1000 and J = 100

In [20]:
# Type In Your Own gurobi.lic
params = {
    "WLSACCESSID": 'YOUR WLS ID',
    "WLSSECRET": 'YOUR WLS SECRET',
    "LICENSEID": int('YOUR LICENSE'),
}
lp_instance_L = lp(N = 1000, J = 100, gp_params = params, min_order_at_full = 0)
m, xvar, yvar, zvar, cost, order_part = lp_instance_L.lp_model()


------Generating Dicts------
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2482374
Academic license 2482374 - for non-commercial use only - registered to zt___@columbia.edu
****** Creating Variable List ******


100%|██████████| 1000/1000 [00:00<00:00, 1226.91it/s]


****** Setting Objective ******


100%|██████████| 1000/1000 [00:00<00:00, 21385.33it/s]


****** Setting Constraints ******


100%|██████████| 19/19 [00:00<00:00, 23.79it/s]


****** Optimising ******
Set parameter TimeLimit to value 10800
Set parameter NodefileStart to value 10
Set parameter Threads to value 8
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2482374 - for non-commercial use only - registered to zt___@columbia.edu
Optimize a model with 21120 rows, 205001 columns and 965001 nonzeros
Model fingerprint: 0xbe19ecad
Variable types: 186001 continuous, 19000 integer (19000 binary)
Coefficient statistics:
  Matrix range     [3e-03, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e-03, 1e+05]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 204441.24506
Presolve removed 1053 rows and 57000 columns
Presolve time: 2.51s
Presolved: 20067 rows, 148001 columns, 577238 nonzeros
Variable types: 130001 continuous, 18000 inte

In [21]:
lp_instance_L.summary(m, xvar, yvar, zvar, cost, order_part)
lp_instance_L.write_dat_file(m, xvar)

Total Cost: 15995.959139352815 

Optimization Gap: 0.00% 

Require 0 orders to be shipped in full 

Actual 1000 orders shipped in full 

0 carriers used at full capacity 

0 plants used at full capacity 



100%|██████████| 185000/185000 [00:01<00:00, 175792.72it/s]


## Extra Credit

In [22]:
# Type In Your Own gurobi.lic
params = {
    "WLSACCESSID": 'YOUR WLS ID',
    "WLSSECRET": 'YOUR WLS SECRET',
    "LICENSEID": int('YOUR LICENSE'),
}
lp_instance_full = lp(N = 1000, gp_params = params, min_order_at_full = 100)
m, xvar, yvar, zvar, cost, order_part = lp_instance_full.lp_model(time_limit=3600)


------Generating Dicts------
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2482374
Academic license 2482374 - for non-commercial use only - registered to zt___@columbia.edu
****** Creating Variable List ******


100%|██████████| 1000/1000 [00:15<00:00, 62.70it/s]


****** Setting Objective ******


100%|██████████| 1000/1000 [00:00<00:00, 22152.70it/s]


****** Setting Constraints ******


100%|██████████| 1000/1000 [00:16<00:00, 61.85it/s]


****** Optimising ******
Set parameter TimeLimit to value 3600
Set parameter NodefileStart to value 10
Set parameter Threads to value 8
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2482374 - for non-commercial use only - registered to zt___@columbia.edu
Optimize a model with 23561 rows, 3923001 columns and 23454001 nonzeros
Model fingerprint: 0xa68a0776
Variable types: 3903001 continuous, 20000 integer (20000 binary)
Coefficient statistics:
  Matrix range     [3e-03, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e-03, 1e+05]
  RHS range        [5e-01, 1e+06]
Presolve removed 46 rows and 0 columns (presolve time = 5s) ...
Presolve removed 46 rows and 0 columns (presolve time = 11s) ...
Presolve removed 46 rows and 58551 columns (presolve time = 15s) ...
Presolve remov

In [23]:
lp_instance_full.summary(m, xvar, yvar, zvar, cost, order_part)
lp_instance_full.write_dat_file(m, xvar)

Total Cost: 15584.362648090268 

Optimization Gap: 0.00% 

Require 100 orders to be shipped in full 

Actual 1000 orders shipped in full 

0 carriers used at full capacity 

0 plants used at full capacity 



100%|██████████| 3902000/3902000 [00:22<00:00, 170793.67it/s]
